# **Final Project Submission**
* Daniel Akwabi
* Part time
* 29/04/2025 at 11:59 PM
* Maryann Mwikali
* Blog post URL: [Git](https://github.com/daniel-akwabi)

# **Aircraft Risk Analysis**

## *Prepared for Pearly Airlines Aviation Department*

### *Overview*
This project aims to precisely assess which aircraft models that present the lowest risk as Pearsly enters the aviation industry. This model will establish the risk levels by deriving data from the "AviationData.csv" dataset that entails civil aviation accidents and selected incidents.

### *Understanding the Problem*
Pearsly is diversifying into aviation operations and needs to identify the lowest risk aircraft models for acquisition as the company ventures into the aviation industry.
The company needs a data driven risk assessment framework to evaluate the models, ensuring informed decisons that are in accordance with its goals while alleviating financial, operational and safety risks.

### *Key Objectives*

- Identify the safest aircraft models based on historical data. The accident rate, fatality rate and incident severity are our key performance indicators in this instance.

- Assess the financial risks. As we work to acheive this, I will focus on the maintenance cost and fuel efficiency of each model in order for me to narrow down to the best fit.

- Actinable acquisition recommendations. Factors such as safety, cost and reliability will assist me in establishing the ideal model(s) for the company's new venture.

### *Outcomes*

As I delve into this research, I expect to have the following outcomes:

- Risk Scoring Model : Jupyter notebook will assist me in assessing the risk levels of each aircraft model, this requires attention on the safety and reliability.

- Strategic Impact : A look at both short term and long term implications will be analyzed in order to establish the purchase decision, insurance premium, fleet risk profile as well as maintenance cost savings.

- Visual Summary : A comprehensive report entailing summarized methodology, findings and recommendations to facilitate further research and expansion of the company. 

### *Hypothesis about risk factors*

Aircraft risk is primarily driven by three measurable factors, that is: safety record, operational costs, and reliability metrics. Newer models with composite materials and digital systems will demonstrate lower risk than the conventional aircraft.

#### *Data Sources*

- FAA/NTSB accident reports  
- Manufacturer maintenance logs  
- Industry benchmarking data .

### **1.Data Loading and Initial Exploration**

In [ ]:
#Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loading data from CSV file

df = pd.read_csv('data\Aviation_Data.csv')

#Displaying basic info
print(df.shape)
df.head()

### *Initial Data Summary*

#### *Purpose of Checking for Missing Values in This Dataset*

Checking for missing values in this aviation accident dataset serves several important purposes:

1. *Data Quality Assessment*: The output shows many columns have significant missing data (e.g., Latitude/Longitude missing ~55,000 values, Aircraft.Category missing ~58,000 values). This helps understand how complete and reliable the dataset is.

2. *Analysis Planning*: Knowing where data is missing helps determine:
   - Which analyses are feasible (columns with too much missing data may need to be excluded)
   - Whether imputation methods are needed
   - How to handle cases with partial information

3. *Bias Identification*: Missing data patterns may reveal systemic reporting issues (e.g., certain countries or accident types might be under-reported).

4. *Data Cleaning Decisions*: The missing values in key fields like Event.Id, Accident.Number (~1,500 missing) and Location (~500 missing) suggest these records may need special handling.

5. *Feature Selection*: For modeling purposes, columns with excessive missing values (like Aircraft.Category with 64% missing) might be poor candidates as features.

This initial check is crucial before performing any substantive analysis to understand the limitations of the dataset and plan appropriate data handling strategies.

In [ ]:
#Check for missing values and data types
df.info()
#Statistical summary 
df.describe()

### *Missing Data Visualization*

In [ ]:
df.isnull().sum().sort_values(ascending=False).plot(kind='bar', 
figsize=(10, 4),
title= "Missing Data Bar Chart", ylabel= "Count") 
plt.show()

### **2.Data Cleaning**

In [ ]:
# Handling the missing data
num_cols = ['Number.of.Engines', 'Latitude', 'Longitude']

for col in num_cols:
    # Ensure numeric type
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Skip if still non-numeric after conversion
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Skipping {col} - could not convert to numeric")
        continue
        
    # Impute if needed
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"Imputed {col} with median: {median_val}")

Based on the available columns in my aviation dataset, I will create a comprehensive risk score calculation that leverages the most relevant safety related features, see the structured approach below

In [ ]:
# 1. Create individual risk components (0-1 scale)
risk_factors = {
    'severity_risk': np.where(df['Injury.Severity'].isin(['Fatal', 'Serious']), 1, 0.5),
    'damage_risk': np.where(df['Aircraft.damage'] == 'Destroyed', 1, 
                           np.where(df['Aircraft.damage'] == 'Substantial', 0.7, 0.3)),
    'engine_risk': df['Number.of.Engines'].apply(lambda x: min(x/4, 1)),  # More engines = higher risk
    'location_risk': np.where(df['Country'].isin(['US', 'Russia', 'China']), 0.8, 0.5),  # High-traffic countries
    'unknown_risk': np.where(df['Investigation.Type'] == 'Unknown', 0.7, 0.2)
}

# 2. Calculate weighted risk score (0-100 scale)
weights = {
    'severity_risk': 0.3,
    'damage_risk': 0.25,
    'engine_risk': 0.2,
    'location_risk': 0.15,
    'unknown_risk': 0.1
}

df['risk_score'] = sum(risk_factors[factor] * weights[factor] for factor in risk_factors) * 100

# 3. Categorize risk levels
df['risk_category'] = pd.cut(df['risk_score'],
                            bins=[0, 30, 70, 90, 100],
                            labels=['Low', 'Medium', 'High', 'Critical'])

### *Breakdown*
- **Injury Severity** carries the most weight at 30%
- **Aircraft Damage** is second most important at 25%
- **Engine Count** contributes moderately with 20% 
- **Geographic and investigation factors** have smaller impacts

In [ ]:
# Risk distribution
plt.figure(figsize=(10,5))
df['risk_category'].value_counts().sort_index().plot(kind='bar')
plt.title('Aviation Incident Risk Distribution')
plt.ylabel('Count of Incidents')
plt.show()

# Top high-risk events
high_risk = df.nlargest(5, 'risk_score')[['Event.Id', 'Location', 'risk_score']]
print("Top 5 Highest Risk Incidents:")
print(high_risk.to_markdown())

### *Key Observations*

1. *Uniform High Risk Scores*:
   - All top incidents share identical risk scores (84.5/100)
   - Suggests these events met similar risk criteria thresholds

2. *Geographical Distribution*:
   - 3 Asian locations (Indonesia, South Korea, Afghanistan)
   - 1 Middle Eastern (Dubai)
   - 1 North American (Reno, NV)
   - Indicates global risk distribution

3. *Temporal Pattern*:
   - Incidents span 2008-2013
   - No clear year-to-year trend in this sample

4. *Common Risk Factors* (likely contributing):
   - *Severe injury/fatalities* (Injury.Severity)
   - *Aircraft destroyed* (Aircraft.damage)
   - *High-traffic regions* (Country risk weighting)
   - *Multiple engines* (Number.of.Engines)

### *Outlier Detection*

In [ ]:
sns.boxplot(x=df['risk_score'])
plt.title('Distributon of Risk Scores')
plt.show()

### **3.Exploratory Data Analysis (EDA)**

### *Creating missing column "Safety_Score"*
Since I do not have this particular column, I will derive it from other columns as elaborated below.

In [ ]:
# Calculate components
safety_df = df.groupby('Model').agg({
    'Injury.Severity': lambda x: (x == 'Fatal').mean(),  # Fatality rate
    'Aircraft.damage': lambda x: x.str.contains('Substantial').mean(),  # Damage rate
    'Number.of.Engines': 'mean'  # More engines = safer (generally)
})

# Normalize and weight
safety_df['safety_score'] = (
    0.5 * (1 - safety_df['Injury.Severity']) +  # Fatality avoidance
    0.3 * (1 - safety_df['Aircraft.damage']) +  # Damage avoidance
    0.2 * (safety_df['Number.of.Engines'] / 4)  # Engine redundancy (scaled to max 4)
) * 100

In this instance, I am using the composite safety index to calculate a meaningful **safety_score** using the **NTSB** aviation accident dataset since it incorporates injury, damage, and design factors. It also aligns with NTSB's holistic safety assessment methodology and it provides intuitive scoring, that is, 0 to 100 scale.
 
 I will combine multiple factors with weights as elaborated in the cell below.

In [ ]:
# Calculate components
safety_df = df.groupby('Model').agg({
    'Injury.Severity': lambda x: (x == 'Fatal').mean(),  # Fatality rate
    'Aircraft.damage': lambda x: x.str.contains('Substantial').mean(),  # Damage rate
    'Number.of.Engines': 'mean'  # More engines = safer (generally)
})

# Normalize and weight
safety_df['safety_score'] = (
    0.5 * (1 - safety_df['Injury.Severity']) +  # Fatality avoidance
    0.3 * (1 - safety_df['Aircraft.damage']) +  # Damage avoidance
    0.2 * (safety_df['Number.of.Engines'] / 4)  # Engine redundancy (scaled to max 4)
) * 100

Now let us go ahead and visualize the results by identifying the top 5 **safety_scores** in the dataset.

In [ ]:
top_5 = safety_df.nlargest(5, 'safety_score')
sns.barplot(x=top_5.index, y='safety_score', data=top_5)
plt.title('Safest Aircraft Models')
plt.xticks(rotation=45)

### **4. Recommendations and Conclusion**

### *Top 3 Recommended Aircraft Models*

Based on the available columns, the simple frequency based ranking will be useful in establishing recommendations to Pearly Airlines. Take a look!

In [ ]:
# Count accidents per model, then select models with fewest incidents
recommendations = df['Model'].value_counts().to_frame('accident_count')
top_models = recommendations.sort_values('accident_count').head(3)

In [ ]:
# Group by aircraft model and calculate safety metrics
recommendations = df.groupby('Model').agg({
    'Injury.Severity': lambda x: (x == 'Fatal').mean(),  # Fatality rate
    'Aircraft.damage': lambda x: x.str.contains('Substantial|Destroyed').mean(),  # Damage rate
    'Number.of.Engines': 'mean'  # Engine redundancy
}).reset_index()

# Calculate composite score (higher = safer)
recommendations['safety_score'] = (
    0.6 * (1 - recommendations['Injury.Severity']) +  # 60% weight to avoid fatalities
    0.3 * (1 - recommendations['Aircraft.damage']) +  # 30% weight to avoid damage
    0.1 * (recommendations['Number.of.Engines'] / 4)  # 10% for engine redundancy
)

# Get top 3
top_3 = recommendations.sort_values('safety_score', ascending=False).head(3)

This works because:
1. No missing columns, that means only confirmed columns in my datasets are used.
2. I have used clear logic: weights severity, damage and design factors.
3. The output ranks models by objective metrics.

Now let us confirm our column names.

In [ ]:
print(df.columns.tolist())

Let us visualize our **Top 3 Safest Models**

In [ ]:
sns.barplot(x='Model', y='safety_score', data=top_3)
plt.title('Top 3 Safest Models')
plt.xticks(rotation=45)

#### *Results Interpretation*
Your top 3 safest models are:  
1. *DHC-7-100*  
   - Likely strengths: Proven reliability in regional operations  
2. *747-422*  
   - Likely strengths: Advanced avionics in later 747 variants  
3. *747-130*  
   - Likely strengths: Robust design from early 747 iterations  

#### *Business Implications*
- *Acquisition Priority*: The DHC-7-100 emerges as the safest choice based on my scoring.  
- *Risk Profile*: All three models have safety scores significantly above my dataset average (visible if I added a reference line).


  #### *Limitations*
- *Scope*: Results are relative to my dataset's time period and reporting standards.  
- *Missing Factors*: Does not account for:  
  - Fleet size (more common models may have inflated accident counts)  
  - Operational context (e.g., cargo vs passenger use)  

---

### *Markdown Table*
| Model | Safety Score | Key Strength | Suggested Use Case |
|-------|--------------|--------------|--------------------|
| DHC-7-100 | Highest | Regional reliability | Short-haul routes |
| 747-422 | High | Modern systems | Long-haul passenger |
| 747-130 | Moderate | Proven airframe | Cargo operations |

---

### *Actionable Next Steps*
- *Validate Findings*: Cross-check with manufacturer safety reports.
- Initiate purchase negotiations for *DHC-7-100*.
- Develop phased implementation plan.